In [ ]:
import speech_recognition as sr
import pyttsx3
import requests
import wikipedia
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
intent_labels = ["weather", "wikipedia"]

def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            return recognizer.recognize_google(audio).lower()
        except sr.UnknownValueError:
            return "Sorry, I couldn't understand that."
        except sr.RequestError:
            return "API request error."

def predict_intent(text):
    result = classifier(text, intent_labels)
    return result["labels"][0]

def get_weather(city):
    API_KEY = "" # GET YOU API KEY FROM OPENWEATHERMAP WEB APPLICATION
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url).json()
    
    if response.get("cod") != 200:
        return "Sorry, I couldn't find weather data."
    
    weather = response["weather"][0]["description"]
    temperature = response["main"]["temp"]
    return f"The weather in {city} is {weather} with a temperature of {temperature}°C."

def get_wikipedia_summary(query):
    try:
        return wikipedia.summary(query, sentences=2)
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Multiple results found: {e.options[:3]}"
    except wikipedia.exceptions.PageError:
        return "I couldn't find that information."
    except Exception:
        return "I couldn't find that information."

def collect_data(user_input, detected_intent):
    with open("training_data.txt", "a") as file:
        file.write(f"{user_input} -> {detected_intent}\n")
    print("Data saved for fine-tuning.")

def main():
    speak("Hello! How can I assist you?")
    
    while True:
        command = listen()
        print("User said:", command)

        if any(exit_word in command for exit_word in ["exit", "bye", "stop", "quit"]):
            speak("Goodbye! Have a great day.")
            break

        intent = predict_intent(command)

        if intent == "weather":
            speak("Which city's weather?")
            city = listen()
            result = get_weather(city)

        else:  # Default to Wikipedia
            result = get_wikipedia_summary(command)

        collect_data(command, intent)

        speak(result)
        print(result)

if __name__ == "__main__":
    main()
